读取文件夹的生成图片，计算IS

In [ ]:
#改变当前工作目录到谷歌云盘的路径
from google.colab import drive
drive.mount('/content/drive/')
import os
path="/content/drive/My Drive/Colab Notebooks/"
os.chdir(path)
os.listdir(path)

Mounted at /content/drive/


['Untitled5.ipynb',
 'BRAIN-LARGE-scVI.ipynb',
 'BRAIN-SMALL.ipynb',
 'BRAIN-LARGE.ipynb',
 'Untitled (1)',
 'BRAIN-LARGE-prepro.ipynb',
 'learn_scanpy.ipynb',
 'Untitled0.ipynb',
 'scvi_RETINA.ipynb',
 '“preprocessing.ipynb”的副本',
 'Untitled1.ipynb',
 'scvi_CORTEX.ipynb',
 'reproduce_scvi_tutorial.ipynb',
 'scScope.ipynb',
 '“ae_mnist_conv_jax.ipynb”的副本 (1)',
 '“ae_mnist_conv_jax.ipynb”的副本',
 '““ae-dae_on_mnist”的副本”的副本',
 '“ae_dae_on_fashionmnist”的副本',
 'Untitled',
 'cauculate_p.ipynb',
 'Untitled2.ipynb',
 'Untitled3.ipynb',
 'self_ae.ipynb',
 'self_vae.ipynb',
 'data',
 'new_IS.ipynb',
 'my_gan.ipynb',
 'my_vae.ipynb',
 'my_dae.ipynb',
 'my_diffusion.ipynb',
 'my_sae.ipynb',
 'my_ae.ipynb',
 'IS.ipynb',
 'FID.ipynb']

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
import torch.utils.data
from torchvision.models.inception import inception_v3
import numpy as np
from tqdm import tqdm
from PIL import Image
import os
from scipy.stats import entropy


mean_inception = [0.485, 0.456, 0.406]
std_inception = [0.229, 0.224, 0.225]

def imread(filename):
    """
    Loads an image file into a (height, width, 3) uint8 ndarray.
    """
    return np.asarray(Image.open(filename), dtype=np.uint8)[..., :3]

def inception_score(batch_size=50, resize=True, splits=10):

    device = torch.device("cuda:0")
    inception_model = inception_v3(pretrained=True, transform_input=False).to(device)
    inception_model.eval()
    up = nn.Upsample(size=(299, 299), mode='bilinear', align_corners=False).to(device)

    def get_pred(x):
        if resize:
            x = up(x)
        x = inception_model(x)
        return F.softmax(x, dim=1).data.cpu().numpy()


    print('Computing predictions using inception v3 model')


    files = readDir()
    N = len(files)
    preds = np.zeros((N, 1000))
    if batch_size > N:
        print(('Warning: batch size is bigger than the data size. '
                 'Setting batch size to data size'))

    for i in tqdm(range(0, N, batch_size)):
        start = i
        end = i + batch_size
        images = np.array([imread(str(f)).astype(np.float32)
                           for f in files[start:end]])


        images = images.transpose((0, 3, 1, 2))
        images /= 255

        batch = torch.from_numpy(images).type(torch.FloatTensor)
        batch = batch.to(device)
        y = get_pred(batch)
        print(y.shape)
        preds[i :i  + batch_size] = get_pred(batch)


    assert batch_size > 0
    assert N > batch_size
    print('Computing KL Divergence')
    split_scores = []
    for k in range(splits):
        part = preds[k * (N // splits): (k + 1) * (N // splits), :] # split the whole data into several parts
        py = np.mean(part, axis=0)
        scores = []
        for i in range(part.shape[0]):
            pyx = part[i, :]
            scores.append(entropy(pyx, py))
        split_scores.append(np.exp(scores))

    return np.max(split_scores), np.mean(split_scores)

def readDir():
    dirPath = r"/content/drive/My Drive/diffusion_fashionmnist_images/"
    allFiles = []
    if os.path.isdir(dirPath):
        fileList = os.listdir(dirPath)
        for f in fileList:
            f = dirPath+'/'+f
            if os.path.isdir(f):
                subFiles = readDir(f)
                allFiles = subFiles + allFiles
            else:
                allFiles.append(f)
        return allFiles
    else:
        return 'Error,not a dir'

MAX, IS= inception_score(splits=10)
print('MAX IS is %.4f' % MAX)
print('The IS is %.4f' % IS)


Computing predictions using inception v3 model


  0%|          | 0/13 [00:00<?, ?it/s]

(50, 1000)


  8%|▊         | 1/13 [00:11<02:14, 11.22s/it]

(50, 1000)


...

(50, 1000)


 85%|████████▍ | 11/13 [00:16<00:01,  1.59it/s]

(50, 1000)


100%|██████████| 13/13 [00:17<00:00,  1.35s/it]

(32, 1000)
Computing KL Divergence
MAX IS is 29.5494
The IS is 2.4168
